In [ ]:
#check requirements.txt for list of necessary libraries to run this script

In [ ]:
import tensorflow as tf
from tqdm import tqdm
import torch
from torch import optim
import torch.nn as nn
import os
from PIL import Image
from torchvision import transforms
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from sklearn.metrics import confusion_matrix

In [ ]:
#Operacje architektury U-net
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_op = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv_op(x)


class DownSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = DoubleConv(in_channels, out_channels)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        down = self.conv(x)
        p = self.pool(down)

        return down, p


class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.up = nn.ConvTranspose2d(in_channels, in_channels//2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
       x1 = self.up(x1)
       x = torch.cat([x1, x2], 1)
       return self.conv(x)

In [ ]:
#set up U-Net
class UNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super().__init__()
        self.down_convolution_1 = DownSample(in_channels, 64)
        self.down_convolution_2 = DownSample(64, 128)
        self.down_convolution_3 = DownSample(128, 256)
        self.down_convolution_4 = DownSample(256, 512)

        self.bottle_neck = DoubleConv(512, 1024)

        self.up_convolution_1 = UpSample(1024, 512)
        self.up_convolution_2 = UpSample(512, 256)
        self.up_convolution_3 = UpSample(256, 128)
        self.up_convolution_4 = UpSample(128, 64)
        
        self.out = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=1)

    def forward(self, x):
       down_1, p1 = self.down_convolution_1(x)
       down_2, p2 = self.down_convolution_2(p1)
       down_3, p3 = self.down_convolution_3(p2)
       down_4, p4 = self.down_convolution_4(p3)

       b = self.bottle_neck(p4)

       up_1 = self.up_convolution_1(b, down_4)
       up_2 = self.up_convolution_2(up_1, down_3)
       up_3 = self.up_convolution_3(up_2, down_2)
       up_4 = self.up_convolution_4(up_3, down_1)

       out = self.out(up_4)
       return out

In [ ]:
#dataset class, used to move across the folder with images of lakes/lake, change paths in accordance with path example in comment within if statement below
#in order to change file directories to move across, reccommended image resolution is 512 by 512 pixels for minimal loss of accuracy
#all images will be resized to this resolution otherwise which may result in a varried amount of loss of accuracy
class mldsjeziora(Dataset):
    def __init__(self, root_path, test=False):
        self.root_path = root_path
        if test:
            #self.images = sorted([root_path+"/test/image/"+i for i in os.listdir(root_path+"/test/image/")])
            #self.masks = sorted([root_path+"/test/masks/"+i for i in os.listdir(root_path+"/test/masks/")])
            self.images = sorted([root_path+"your_image_folder"+i for i in os.listdir(root_path+"your_image_folder")])
            self.masks = sorted([root_path+"your_mask_folder"+i for i in os.listdir(root_path+"your_mask_folder")])
        else:
            self.images = sorted([root_path+"your_image_folder"+i for i in os.listdir(root_path+"your_image_folder")])
            self.masks = sorted([root_path+"your_mask_folder"+i for i in os.listdir(root_path+"your_mask_folder")])
        
        self.transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor()])

    def __getitem__(self, index):
        img = Image.open(self.images[index]).convert("L")
        img = Image.open(self.images[index]).convert("RGB")
        mask = Image.open(self.masks[index]).convert("L")

        return self.transform(img), self.transform(mask)

    def __len__(self):
        return len(self.images)

In [ ]:
#sum class used to calculate pixel based surface area of the lake from the mask/model output
def summation(test_tup):
 
    # Converting into list
    test = list(test_tup)
 
    # Initializing count
    count = 0
 
    # for loop
    for i in test:
        count += i
    return count

In [ ]:
#used to check images with the model and compare with test data prepared ahead of time
def pred_show_image_grid(data_path, model_pth, device):
    model = UNet(in_channels=3, num_classes=1).to(device)
    model.load_state_dict(torch.load(model_pth, weights_only=True, map_location=torch.device(device)))
    image_dataset = mldsjeziora(data_path, test=True)
    images = []
    orig_masks = []
    pred_masks = []
    total_lake_sa = 0
    total_mask_sa = 0
    for img, orig_mask in image_dataset:
        img = img.float().to(device)
        img = img.unsqueeze(0)

        pred_mask = model(img)

        img = img.squeeze(0).cpu().detach()
        img = img.permute(1, 2, 0)
        
        pred_mask = pred_mask.squeeze(0).cpu().detach()
        pred_mask = pred_mask.permute(1, 2, 0)
        
        pred_mask[pred_mask < 0.81]=0
        pred_mask[pred_mask > 0.81]=1
        
        orig_mask = orig_mask.cpu().detach()
        orig_mask = orig_mask.permute(1, 2, 0)

        images.append(img)
        orig_masks.append(orig_mask)
        pred_masks.append(pred_mask)

        lake_sa = sum(pred_mask)
        total_lake_sa = total_lake_sa + lake_sa
        orig_mask_sa = sum(orig_mask)
        total_mask_sa = total_mask_sa + orig_mask_sa
    
    images.extend(orig_masks)
    images.extend(pred_masks)
    fig = plt.figure()
    for i in range(1, 3*len(image_dataset)+1):
       fig.add_subplot(3, len(image_dataset), i)
       plt.imshow(images[i-1], cmap="gray")
    plt.show()

    #calculations below give surface area measured assuming the input image pixel resolution of n and input image size of 512 by 512
    #other image sizes will make this calculation inaccurate
    #change n in order to adjust pixel resolution within calculation
    n = 0.25
    #total lake surface area summed up from images interpreted by the model in ha
    tlsa = summation(total_lake_sa)*n*n*0.0001
    #total lake surface area based on masks prepared ahead of time in ha
    omsa = summation(total_mask_sa)*n*n*0.0001
    print(tlsa)
    print(omsa)
    #ratio of predicted to known surface area of the lake
    print(tlsa/omsa)

In [ ]:
def calc_lakes(data_path, model_pth, device, pixel_res):
    model = UNet(in_channels=3, num_classes=1).to(device)
    model.load_state_dict(torch.load(model_pth, weights_only=True, map_location=torch.device(device)))
    image_dataset = mldsjeziora_f(data_path)
    lake_sa = []
    for i in os.listdir(data_path):
        images = []
        pred_masks = []
        total_lake_sa = 0
        
    for img in image_dataset:
        img = img.float().to(device)
        img = img.unsqueeze(0)

        pred_mask = model(img)

        img = img.squeeze(0).cpu().detach()
        img = img.permute(1, 2, 0)
        
        pred_mask = pred_mask.squeeze(0).cpu().detach()
        pred_mask = pred_mask.permute(1, 2, 0)
        
        pred_mask[pred_mask < 0.81]=0
        pred_mask[pred_mask > 0.81]=1

        images.append(img)
        pred_masks.append(pred_mask)
        pred_mask = pred_mask.numpy()
        lake_sa = sum(pred_mask)
        total_lake_sa = total_lake_sa + lake_sa
        #total lake surface area summed up from images interpreted by the model in ha
        tlsa = summation(total_lake_sa)*pixel_res*pixel_res*0.0001
    var1 = tlsa.tolist()
    var2 = tlsa[0]
    
    
    images.extend(pred_masks)
    fig = plt.figure()
    for i in range(1, 2*len(image_dataset)+1):
       fig.add_subplot(2, len(image_dataset), i)
       plt.imshow(images[i-1], cmap="gray")
    plt.show()
    
    return var2
    

In [ ]:
#dataset class for final dataset without masks, used to move across the folder with images of lakes/lake, change paths in accordance with path example in comment within if statement below
#in order to change file directories to move across, reccommended image resolution is 512 by 512 pixels for minimal loss of accuracy
#all images will be resized to this resolution otherwise which may result in a varried amount of loss of accuracy
class mldsjeziora_f(Dataset):
    def __init__(self, root_path):
        self.root_path = root_path
        
        #self.images = sorted([root_path+"/test/image/"+i for i in os.listdir(root_path+"/test/image/")])
        self.images = sorted([root_path+i for i in os.listdir(root_path)])
        
        self.transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor()])

    def __getitem__(self, index):
        img = Image.open(self.images[index]).convert("RGB")

        return self.transform(img)

    def __len__(self):
        return len(self.images)

In [ ]:
#used to create a roc curve and jaccard with test data prepared ahead of time, train mode 0
def pred_show_image_grid_scikit(data_path, model_pth, device):
    model = UNet(in_channels=3, num_classes=1).to(device)
    model.load_state_dict(torch.load(model_pth, weights_only=True, map_location=torch.device(device)))
    image_dataset = mldsjeziora(data_path, test=True)
    images = []
    orig_masks = []
    pred_masks = []
    pred_mask_jacc = []
    for img, orig_mask in image_dataset:
        
        img = img.float().to(device)
        img = img.unsqueeze(0)

        pred_mask = model(img)

        img = img.squeeze(0).cpu().detach()
        img = img.permute(1, 2, 0)
        
        pred_mask = pred_mask.squeeze(0).cpu().detach()
        pred_mask = pred_mask.permute(1, 2, 0)
        
        orig_mask = orig_mask.cpu().detach()
        orig_mask = orig_mask.permute(1, 2, 0)

        images.append(img)
        orig_masks.append(orig_mask)
        pred_masks.append(pred_mask)

    y_preds = np.array(pred_masks)
    y_test = np.array(orig_masks)
    
    
    fpr, tpr, roc_auc = roc_plot(y_test, y_preds)
    fig, ax = plt.subplots(1,1)
    ax.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic')
    ax.legend(loc="lower right")
    images.extend(orig_masks)
    images.extend(pred_masks)
    fig = plt.figure()
    for i in range(1, 3*len(image_dataset)+1):
        fig.add_subplot(3, len(image_dataset), i)
        plt.imshow(images[i-1], cmap="gray")
    plt.show()
    jacccard_similarity(y_preds, y_test)

In [ ]:
def jacccard_similarity(y, x):
    x[x>=0.61] = 1
    x[x<0.61] = 0
    x = np.asarray(x, np.bool)
    y = np.asarray(y, np.bool)
    z = np.double(np.bitwise_and(x, y).sum()) / np.double(np.bitwise_or(x, y).sum())
    print("jaccard = ")
    print(z)

In [ ]:
def roc_plot(ground_truth, pred):
    ground_truth_labels = ground_truth.ravel()
    ground_truth_labels = tf.cast(ground_truth_labels,tf.int8)
    score_value = 1-pred.ravel()/255.0
    #false positive rates, true positive rates
    fpr, tpr, thresholds = metrics.roc_curve( ground_truth_labels,score_value)
    roc_auc = metrics.auc(fpr, tpr)
    return fpr, tpr, roc_auc

In [ ]:
if __name__ == "__main__":
    #environment variables changed to make sure tensorflow works correctly in case of no gpu device available
    os.environ['KMP_DUPLICATE_LIB_OK']='True'
    os.environ['TF_ENABLE_ONEDNN_OPTS']='False'

    #file paths
    DATA_PATH = 'your_path_to_directories_containing_machine_learning_data'
    DATA_PATH_PATCHES = "your_path_to_directories_containing_patched_images_for_image_recognition_and_classification"
    MODEL_PATH = 'path_to_your_model'
    MODEL_SAVE_PATH = 'path_where_your_model_will_be_saved'

    #check cuda and set device to gpu mode if gpu available
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    #set train_mode to:
    #0 to get roc curve and jaccard index  
    #1 to resume machine learning process 
    #2 to calculate the dataset
    train_mode = 2

    #calculations below give surface area measured assuming the input image pixel resolution of n and input image size of 512 by 512
    #other image sizes will make this calculation inaccurate
    #change pixel_res[meters] in order to adjust pixel resolution within calculation
    pixel_res = 0.25
    
    if train_mode == 0:
        DATA_PATH = 'path_to_folder_with_test_data'
        pred_show_image_grid_scikit(DATA_PATH, MODEL_PATH, device)
    elif train_mode== 1:
        LEARNING_RATE = 3e-4
        BATCH_SIZE = 5
        EPOCHS = 8
        train_dataset = mldsjeziora(DATA_PATH)
       
        generator = torch.Generator().manual_seed(42)
        train_dataset, val_dataset = random_split(train_dataset,[0.8, 0.2], generator=generator)
    
        train_dataloader = DataLoader(dataset=train_dataset,
                                  batch_size=BATCH_SIZE,
                                  shuffle=True)
        val_dataloader = DataLoader(dataset=val_dataset,
                                batch_size=BATCH_SIZE,
                                shuffle=True)

        model = UNet(in_channels=3, num_classes=1).to(device)
        model.load_state_dict(torch.load(MODEL_PATH, weights_only=True, map_location=torch.device(device)))
        optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
        criterion = nn.BCEWithLogitsLoss()
    
        for epoch in tqdm(range(EPOCHS)):
            model.train()
            train_running_loss = 0
            for idx, img_mask in enumerate(tqdm(train_dataloader)):
                img = img_mask[0].float().to(device)
                mask = img_mask[1].float().to(device)

                y_pred = model(img)
                optimizer.zero_grad()
            
                loss = criterion(y_pred, mask)
                train_running_loss += loss.item()
            
                loss.backward()
                optimizer.step()

            train_loss = train_running_loss / (idx + 1)

            model.eval()
            val_running_loss = 0
            with torch.no_grad():
                for idx, img_mask in enumerate(tqdm(val_dataloader)):
                    img = img_mask[0].float().to(device)
                    mask = img_mask[1].float().to(device)
                
                    y_pred = model(img)
                    loss = criterion(y_pred, mask)

                    val_running_loss += loss.item()
    
                val_loss = val_running_loss / (idx + 1)

            print("-"*30)
            print(f"Train Loss EPOCH {epoch+1}: {train_loss:.4f}")
            print(f"Valid Loss EPOCH {epoch+1}: {val_loss:.4f}")
            print("-"*30)

        torch.save(model.state_dict(), MODEL_SAVE_PATH)

    elif train_mode == 2:
        limit = 0
        lake_sa = []
        lake_dir = []
        for i in os.listdir(DATA_PATH_PATCHES):
            directory = str(DATA_PATH_PATCHES + i)
            lake_dir.append(directory)
            print(directory)
            var = calc_lakes((DATA_PATH_PATCHES + i + "/"), MODEL_PATH, device, pixel_res)
            print(var)
            lake_sa.append(var)
            limit +=1
            print(limit)
        print(lake_sa)
        print(lake_dir)
        np.savetxt("specify_file_name_and_path_to_save_the_measured_surface_and_which_directory's_files_were_used_for_it",[p for p in zip(lake_dir, lake_sa)] , delimiter=',', fmt='%s')